In [1]:
import os, math, numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [3]:
# %%time
# !pip uninstall -y torch
# !pip install -U --no-index --find-links=/kaggle/input/vllm-whl -U vllm
# !pip install -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl

In [4]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 79.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# # In DEBUG mode, infer only on 5 problems
# DEBUG = False
# # Number of candidate solutions to generate
# K = 5
# DEPTH = 4
# TEMPERATURE = 0.8
# TOP_P = 0.8
# BATCH_SIZE = 64
# # C_K = 5 # no of context for a problem

In [5]:
import vllm
import re
import csv
import torch
import gc
from tqdm import tqdm
import pandas as pd
from queue import Queue, Empty
import os
import re
import signal
import subprocess
import tempfile
from collections import Counter
from contextlib import contextmanager

import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

In [9]:
llm = vllm.LLM(
    "/kaggle/input/gemma-3/transformers/gemma-3-4b-it/1",
    tensor_parallel_size=2, 
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    # dtype="bfloat16", 
    enforce_eager=True,
    max_model_len=4096,
)
tokenizer = llm.get_tokenizer()

WARNING 07-26 04:57:59 [config.py:3392] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float32 for compatibility.
INFO 07-26 04:57:59 [config.py:3437] Upcasting torch.bfloat16 to torch.float32.
INFO 07-26 04:57:59 [config.py:1604] Using max model len 4096
WARNING 07-26 04:57:59 [cuda.py:103] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-26 04:57:59 [llm_engine.py:228] Initializing a V0 LLM engine (v0.10.0) with config: model='/kaggle/input/gemma-3/transformers/gemma-3-4b-it/1', speculative_config=None, tokenizer='/kaggle/input/gemma-3/transformers/gemma-3-4b-it/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float32, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size

/usr/local/lib/python3.11/dist-packages/transformers/models/gemma3/configuration_gemma3.py:242: FutureWarning: The `sliding_window_pattern` attribute is deprecated and will be removed in v4.55.0.
  warnings.warn(


INFO 07-26 04:58:01 [cuda.py:346] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 07-26 04:58:01 [cuda.py:395] Using XFormers backend.


2025-07-26 04:58:05.810652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753505885.831893     189 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753505885.838526     189 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 07-26 04:58:11 [__init__.py:235] Automatically detected platform cuda.
(VllmWorkerProcess pid=189) INFO 07-26 04:58:11 [multiproc_worker_utils.py:226] Worker ready; awaiting tasks
(VllmWorkerProcess pid=189) INFO 07-26 04:58:12 [cuda.py:346] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
(VllmWorkerProcess pid=189) INFO 07-26 04:58:12 [cuda.py:395] Using XFormers backend.


...

(VllmWorkerProcess pid=189) /usr/local/lib/python3.11/dist-packages/transformers/models/gemma3/configuration_gemma3.py:242: FutureWarning: The `sliding_window_pattern` attribute is deprecated and will be removed in v4.55.0.
(VllmWorkerProcess pid=189)   warnings.warn(


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 07-26 04:59:57 [default_loader.py:262] Loading weights took 48.88 seconds
(VllmWorkerProcess pid=189) INFO 07-26 04:59:57 [default_loader.py:262] Loading weights took 49.04 seconds
(VllmWorkerProcess pid=189) INFO 07-26 04:59:58 [model_runner.py:1115] Model loading took 9.1552 GiB and 49.309196 seconds
INFO 07-26 04:59:58 [model_runner.py:1115] Model loading took 9.1552 GiB and 49.080298 seconds


(VllmWorkerProcess pid=189) Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


(VllmWorkerProcess pid=189) WARNING 07-26 05:00:09 [logger.py:71] The sequence length used for profiling (max_num_batched_tokens / max_num_seqs = 256) is too short to hold the multi-modal embeddings in the worst case (261 tokens in total, out of which {'image': 256} are reserved for multi-modal embeddings). This may cause certain multi-modal inputs to fail during inference, even when the input text is short. To avoid this, you should increase `max_model_len`, reduce `max_num_seqs`, and/or reduce `mm_counts`.
WARNING 07-26 05:00:10 [logger.py:71] The sequence length used for profiling (max_num_batched_tokens / max_num_seqs = 256) is too short to hold the multi-modal embeddings in the worst case (261 tokens in total, out of which {'image': 256} are reserved for multi-modal embeddings). This may cause certain multi-modal inputs to fail during inference, even when the input text is short. To avoid this, you should increase `max_model_len`, reduce `max_num_seqs`, and/or reduce `mm_counts`.


In [18]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=1000,
)

In [22]:
test_data = pd.read_csv('/kaggle/input/binary-biplob-can-you-decode-emotions/bangla/test.csv')
test_data.head()

,id,text
0,sample_798,I bought a নতুন বই to read
1,sample_141,Bondhudero sathe ghurte giyechilam
2,sample_675,Bazare aj onek bhir chilo
3,sample_574,এই movie টা really interesting ছিল
4,sample_488,সন্ধ্যায় পার্কে হাঁটতে যাবো


In [24]:
sentences = test_data['text'].tolist()
sentences[:5]

['I bought a নতুন বই to read',
 'Bondhudero sathe ghurte giyechilam',
 'Bazare aj onek bhir chilo',
 'এই movie টা really interesting ছিল',
 'সন্ধ্যায় পার্কে হাঁটতে যাবো']

In [25]:
sys_prompt = '''You are a multilingual language model who understands Bengali, English, and Banglish (a mixture of Bengali and English).
Your task is to analyze the sentiment of the below sentence and answer in sentiment category ('negative', 'positive', 'neutral').
You should think step by step and give me the answer in the box like {{positive}}.'''

# Create full prompts by combining system prompt with sentence
prompts = [f"{sys_prompt}\nSentence: {sentence}" for sentence in sentences]

In [27]:
prompts[:2]

["You are a multilingual language model who understands Bengali, English, and Banglish (a mixture of Bengali and English).\nYour task is to analyze the sentiment of the below sentence and answer in sentiment category ('negative', 'positive', 'neutral').\nYou should think step by step and give me the answer in the box like {{positive}}.\nSentence: I bought a নতুন বই to read",
 "You are a multilingual language model who understands Bengali, English, and Banglish (a mixture of Bengali and English).\nYour task is to analyze the sentiment of the below sentence and answer in sentiment category ('negative', 'positive', 'neutral').\nYou should think step by step and give me the answer in the box like {{positive}}.\nSentence: Bondhudero sathe ghurte giyechilam"]

In [28]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=2000,
)

# Generate predictions
results = llm.generate(prompts, sampling_params)

# Extract results
outputs = [res.outputs[0].text.strip() for res in results]

Adding requests:   0%|          | 0/120 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/120 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 07-26 05:25:33 [scheduler.py:1822] Sequence group 136 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


In [29]:
outputs[:5]

['.\nStep 1: Analyze the words in the sentence. "নতুন" (new) is generally positive. "বই" (book) is an object, so it\'s neutral. "to read" suggests an activity, which can be positive or neutral depending on the context.\nStep 2: Considering the overall context, the sentence expresses a positive action - buying a new book to read.\nStep 3: Therefore, the sentiment of the sentence is positive.\n\n{{',
 ', ami dukkhito hotoi.\n\nHere is how you should analyze the sentence:\n1. Understand the meaning of the sentence.\n2. Identify the words that express sentiment.\n3. Determine the overall sentiment.\n\nSentence: Bondhudero sathe ghurte giyechilam, ami dukkhito hotoi.\n1. The sentence means "I went with friends, I am feeling sad."\n2. The word "dukkito" (sad) expresses',
 '.\nLet\'s analyze the sentence.\nThe sentence "Bazare aj onek bhir chilo" in Bengali translates to "Today there are many shops in the market."\nThe words "onek bhir" (many shops) generally have a positive connotation as th

In [30]:
import re
def extract_sentiment(text):
    match = re.search(r"\{\{(positive|negative|neutral)\}\}", text.lower())
    return match.group(1) if match else "unknown"

In [31]:
predicted = [extract_sentiment(res.outputs[0].text.strip()) for res in results]

In [32]:
# Save to submission.csv format
submission = pd.DataFrame({
    "id": test_data["id"],
    "label": predicted
})
submission.head()

,id,label
0,sample_798,unknown
1,sample_141,unknown
2,sample_675,positive
3,sample_574,positive
4,sample_488,positive


In [33]:
submission.to_csv("submission.csv", index=False)